In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials

from tastytrade.sessions.enumerations import Channels
from tastytrade.sessions.sockets import DXLinkManager

from datetime import datetime, timedelta

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

start_time = datetime.now() - timedelta(minutes=30)
end_time = datetime.now() - timedelta(minutes=2)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials)

In [3]:
await dxlink.subscribe_to_candles(
    symbol="BTC/USD:CXTALP",
    # symbol="SPY",
    interval="1m",
    # from_time=datetime.now(),
    from_time=start_time,
    # to_time=end_time,
)

In [ ]:
# Subscribe to 5-minute candles for SPY
await dxlink.unsubscribe_to_candles(symbol="BTC/USD:CXTALP", interval="1m")
# await dxlink.unsubscribe_to_candles(symbol="SPY", interval="1m")

In [ ]:
symbols = ["BTC/USD:CXTALP", "SPY", "NVDA"]
await dxlink.subscribe(symbols)

In [ ]:
await dxlink.unsubscribe(symbols)

In [ ]:
dxlink.router.handler[Channels.Candle].processors["feed"].pl

In [ ]:
# Access candle data
columns = ["eventSymbol", "time", "open", "high", "low", "close"]
dxlink.router.handler[Channels.Candle].processors["feed"].df[columns]
# dxlink.router.handler[Channels.Candle].processors["feed"].pl.select(columns)

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Greeks].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Profile].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Summary].processors["feed"].df

In [ ]:
await dxlink.close()

In [4]:
import plotly.graph_objects as go
from IPython.display import display, clear_output
import asyncio
import logging


def plot_live_candlesticks(dxlink, symbol):
    """Live candlestick chart that updates based on the most recent data.

    Args:
        dxlink: DXLink manager instance
        symbol: Symbol to plot (e.g. "BTC/USD:CXTALP{=5m}")
    """
    logger = logging.getLogger(__name__)
    fig = go.Figure()

    # Set up the figure style
    fig.update_layout(
        plot_bgcolor="rgb(25,25,25)",
        paper_bgcolor="rgb(25,25,25)",
        title=dict(text=symbol, x=0.5, font=dict(color="white", size=16)),
        yaxis=dict(
            gridcolor="rgba(128,128,128,0.1)", zerolinecolor="rgba(128,128,128,0.1)", color="white"
        ),
        xaxis=dict(
            gridcolor="rgba(128,128,128,0.1)", zerolinecolor="rgba(128,128,128,0.1)", color="white"
        ),
        showlegend=False,
    )

    async def update_chart():
        try:
            while True:
                # Get data
                df = (
                    dxlink.router.handler[Channels.Candle]
                    .processors["feed"]
                    .df.loc[lambda x: x["eventSymbol"] == symbol]
                )

                logger.info(f"Got dataframe with {len(df)} rows")

                if len(df) == 0:
                    logger.warning("No data available for symbol %s", symbol)
                    await asyncio.sleep(1)
                    continue

                # Convert timestamps to EDT
                df = df.copy()
                df["time"] = df["time"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")

                # Update the candlesticks
                try:
                    with fig.batch_update():
                        fig.data = []
                        fig.add_trace(
                            go.Candlestick(
                                x=df["time"],
                                open=df["open"],
                                high=df["high"],
                                low=df["low"],
                                close=df["close"],
                                increasing=dict(line=dict(width=1), fillcolor="#26A69A"),
                                decreasing=dict(line=dict(width=1), fillcolor="#EF5350"),
                            )
                        )

                    clear_output(wait=True)
                    display(fig)
                    logger.info("Plot updated successfully")

                except Exception as e:
                    logger.error(f"Error updating plot: {e}")

                # Wait for next update
                await dxlink.queues[Channels.Candle.value].get()

        except asyncio.CancelledError:
            logger.info(f"Stopped plotting {symbol}")
        except Exception as e:
            logger.error(f"Unexpected error in chart update: {e}")

    return asyncio.create_task(update_chart(), name=f"candlestick_plot_{symbol}")

In [ ]:
task = plot_live_candlesticks(dxlink, "BTC/USD:CXTALP{=m}")

In [ ]:
task.cancel()

In [ ]:
await dxlink.close()